In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
%pip install git+https://github.com/cocodataset/panopticapi.git
!git clone https://github.com/bytedance/fc-clip.git

In [ ]:
%cd fc-clip
%pip install -r requirements.txt

In [ ]:
%cd fcclip/modeling/pixel_decoder/ops

In [ ]:
!sh make.sh

In [ ]:
%cd ../../../..

In [ ]:
%pip install open-clip-torch==2.24.0

In [ ]:
import os
import torch
import detectron2
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.data.datasets import load_coco_json
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.evaluation import COCOEvaluator
from fcclip import (
    InstanceSegEvaluator,
    MaskFormerInstanceDatasetMapper,
    add_maskformer2_config,
    add_fcclip_config,
)

In [20]:
PROMPTS = [
    "A discarded {} found on the ground.",
    "A {} lying among scattered trash.",
    "A {} left as litter in the street.",
    "A photo of a {} thrown away in a public space.",
    "A crumpled {} lying near other garbage.",
    "A {} mixed with other debris on the ground.",
    "A broken {} discarded in the environment.",
    "This is a {} among other littered objects.",
    "A small {} found discarded in a cluttered area.",
    "A {} partially covered by other trash in the scene.",
    "A large {} lying near a pile of garbage.",
    "A weathered {} discarded on the side of the road.",
    "A crushed {} among other waste on the street.",
    "A {} that has been thrown away and left as litter.",
    "A {} carelessly discarded in a public park.",
    "A {} found among a variety of litter in this scene.",
    "A {} thrown into the corner of a littered area.",
    "This is a {} abandoned as trash in the environment.",
    "A {} lying in a heap of litter, partially hidden.",
    "A photo of a {} found on the sidewalk among other debris."
]

In [9]:
data_dir_path = "/content/drive/MyDrive/instseg/data/"

In [10]:
def register_coco_dataset(name, json_file, image_root):
    detectron2.data.DatasetCatalog.register(
        name,
        lambda: load_coco_json(json_file, image_root, name),
    )
    detectron2.data.MetadataCatalog.get(name).set(
        thing_classes=[
            "Can",
            "Other",
            "Bottle",
            "Bottle cap",
            "Cup",
            "Lid",
            "Plastic bag + wrapper",
            "Pop tab",
            "Straw",
            "Cigarette",
        ]
    )

In [11]:
def configure_model():
    cfg = detectron2.config.get_cfg()
    add_deeplab_config(cfg)
    add_maskformer2_config(cfg)
    add_fcclip_config(cfg)
    cfg.merge_from_file("configs/coco/panoptic-segmentation/fcclip/fcclip_convnext_large_eval_coco.yaml")
    cfg.MODEL.WEIGHTS = (
        "/content/drive/MyDrive/instseg/fcclip_cocopan.pth"
    )
    cfg.MODEL.MASK_FORMER.TEST.INSTANCE_ON = True
    cfg.MODEL.MASK_FORMER.TEST.SEMANTIC_ON = False
    cfg.MODEL.MASK_FORMER.TEST.PANOPTIC_ON = False
    cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 10
    cfg.DATASETS.TEST = ("taco10_test",)
    cfg.MODEL.DEVICE = "cuda"
    cfg.freeze()
    return cfg

In [12]:
register_coco_dataset(
        name="taco10_test",
        json_file=f"{data_dir_path}mapped_annotations_0_test.json",
        image_root=f"{data_dir_path}images/",
)

In [13]:
detectron2.data.MetadataCatalog.get("taco10_test").set(
    json_file=f"{data_dir_path}mapped_annotations_0_test.json"
)

namespace(name='taco10_test',
          thing_classes=['Can',
                         'Other',
                         'Bottle',
                         'Bottle cap',
                         'Cup',
                         'Lid',
                         'Plastic bag + wrapper',
                         'Pop tab',
                         'Straw',
                         'Cigarette'],
          json_file='/content/drive/MyDrive/instseg/data/mapped_annotations_0_test.json')

In [14]:
cfg = configure_model()

In [15]:
predictor = DefaultPredictor(cfg)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/open_clip/factory.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fli

In [16]:
evaluator = InstanceSegEvaluator("taco10_test",output_dir="./output")

In [17]:
test_loader = build_detection_test_loader(cfg, dataset_name="taco10_test", mapper=MaskFormerInstanceDatasetMapper(cfg, is_train=False))

In [18]:
evaluation_results = inference_on_dataset(predictor.model, test_loader, evaluator)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.

In [19]:
print(evaluation_results["segm"])

{'AP': 5.715046577724339, 'AP50': 9.616931490558422, 'AP75': 5.102638184253004, 'APs': 0.0, 'APm': 10.538036380737848, 'APl': 8.537141711059494, 'AP-Can': 3.7312812252885212, 'AP-Other': 1.1523077820832237, 'AP-Bottle': 26.328826029125587, 'AP-Bottle cap': 1.3813294476712445, 'AP-Cup': 17.55552176070476, 'AP-Lid': 0.935888339441007, 'AP-Plastic bag + wrapper': 4.9035436464667415, 'AP-Pop tab': 0.0, 'AP-Straw': 0.07265863557122612, 'AP-Cigarette': 1.089108910891089}
